In [8]:
import os
import json
import warnings
import folium
from geopy.geocoders import Nominatim
from duckduckgo_search import DDGS
import pypdf
import docx
from typing import TypedDict
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

# Suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, module="duckduckgo_search")
warnings.filterwarnings("ignore", category=ResourceWarning)

# Configuration
MODEL_NAME = "llama3.1:8b" 
MEMORY_FILE = "user_profile.json"
MAP_FOLDER = "generated_maps"

if not os.path.exists(MAP_FOLDER):
    os.makedirs(MAP_FOLDER)

# Helper Functions

def _save_to_profile(key, value):
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'r') as f:
            data = json.load(f)
    else:
        data = {}
    data[key] = value
    with open(MEMORY_FILE, 'w') as f:
        json.dump(data, f, indent=4)
    return f"Saved {key}: {value}"

def _generate_map(city_name):
    geolocator = Nominatim(user_agent="career_agent_v2")
    try:
        location = geolocator.geocode(city_name)
        if location:
            m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)
            folium.Marker([location.latitude, location.longitude], popup=f"{city_name}").add_to(m)
            filename = f"{city_name.lower().replace(' ', '_')}_map.html"
            filepath = os.path.join(MAP_FOLDER, filename)
            m.save(filepath)
            return f"Map saved at: {filepath}"
        return "Map Error: City not found."
    except Exception as e:
        return f"Map Error: {e}"

# Tool Definitions

@tool
def update_user_details(name: str = None, skills: str = None, experience_level: str = None):
    """
    Useful for saving user details when they introduce themselves.
    """
    print(f"Tool Log: Updating User Details...")
    status = []
    if name: status.append(_save_to_profile('name', name))
    if skills: status.append(_save_to_profile('skills', skills))
    if experience_level: status.append(_save_to_profile('experience', experience_level))
    
    return "\n".join(status) if status else "No details updated."

@tool
def career_toolbox(role: str, location: str, save_goal: bool = True):
    """
    Tool Box: Use this for any job search or location request.
    It performs 3 actions:
    1. Saves the career goal.
    2. Searches for jobs on major platforms.
    3. Generates a location map.
    """
    print(f"Tool Log: Executing Tool Box for: {role} in {location}")
    output = []
    
    # 1. Save Goal
    if save_goal:
        _save_to_profile('goal', f"{role} in {location}")
        output.append(f"Goal saved: {role} in {location}")

    # 2. Search Jobs
    try:
        query = f'"{role}" jobs in "{location}" site:linkedin.com OR site:naukri.com OR site:indeed.com'
        results = DDGS().text(query, max_results=4)
        if results:
            output.append("\nTop Job Links:")
            for r in results:
                output.append(f"- {r['title']}: {r['href']}")
        else:
            output.append("\nNo direct job listings found.")
    except Exception as e:
        output.append(f"\nSearch Error: {e}")

    # 3. Map
    map_status = _generate_map(location)
    output.append(f"\n{map_status}")

    return "\n".join(output)

@tool
def read_resume_file(file_path: str):
    """Useful for reading local PDF/DOCX files."""
    print(f"Tool Log: Reading file: {file_path}")
    if not os.path.exists(file_path):
        return "Error: File not found."
    
    text = ""
    try:
        if file_path.endswith('.pdf'):
            reader = pypdf.PdfReader(file_path)
            for page in reader.pages:
                text += page.extract_text() or ""
        elif file_path.endswith('.docx'):
            doc = docx.Document(file_path)
            text = "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        return f"Error reading file: {e}"
    
    return text[:2000]

# Agent Orchestration

class AgentState(TypedDict):
    messages: list

llm = ChatOllama(model=MODEL_NAME, temperature=0)
tools = [update_user_details, career_toolbox, read_resume_file]
llm_with_tools = llm.bind_tools(tools)

def agent_node(state: AgentState):
    messages = state['messages']
    
    # Load profile for context
    profile_text = "{}"
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'r') as f:
            profile_text = json.dumps(json.load(f))

    # System Prompt with Priorities
    system_instruction = f"""You are a helpful Career Assistant.
    Current User Profile: {profile_text}

    YOUR PRIORITY LIST:
    
    1. TOOL RESULTS: If the last message is a Tool Output, summarize it for the user. Do not refuse or apologize.

    2. VALID REQUESTS:
       - Introductions -> Call update_user_details.
       - Job/Location queries -> Call career_toolbox.
       - Resume reading -> Call read_resume_file.

    3. INVALID REQUESTS:
       - Only if no tool applies and the user asks about Coding Scripts, Cooking, or Politics.
       - Say: "I'm sorry, I can only assist with career-related topics."

    Do not over-apologize. Be direct and helpful.
    """
    
    if isinstance(messages[0], SystemMessage):
        messages[0] = SystemMessage(content=system_instruction)
    else:
        messages.insert(0, SystemMessage(content=system_instruction))

    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)

workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")

def should_continue(state: AgentState):
    last_message = state['messages'][-1]
    if last_message.tool_calls:
        return "tools"
    return END

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

app_graph = workflow.compile()

# Execution

def run_chat(user_input):
    print(f"\nUSER: {user_input}")
    initial_state = {"messages": [HumanMessage(content=user_input)]}
    
    for event in app_graph.stream(initial_state):
        for key, value in event.items():
            if key == "agent":
                msg = value["messages"][-1]
                if not msg.tool_calls:
                    print(f"\nAGENT: {msg.content}")
            elif key == "tools":
                pass

if __name__ == "__main__":
    
    # Test 1: Introduction
    run_chat("My name is Rohan and I am an experienced Data Scientist.")
    
    # Test 2: Tool Box
    run_chat("I want to work as a React Developer in Hyderabad. Find jobs and show me the map.")

    # Test 3: Guardrail
    run_chat("Write a Python script to scrape Amazon product prices.")


USER: My name is Rohan and I am an experienced Data Scientist.
Tool Log: Updating User Details...

AGENT: Welcome, Rohan! I'm here to help you with your career-related queries. What's on your mind today?

USER: I want to work as a React Developer in Hyderabad. Find jobs and show me the map.
Tool Log: Executing Tool Box for: React Developer in Hyderabad


C:\Users\hari7\AppData\Local\Temp\ipykernel_4448\116158197.py:91: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=4)



AGENT: Based on the map saved, it seems that you're interested in exploring job opportunities as a React Developer in Hyderabad. I've generated a map that highlights some of the top tech hubs and companies in the city. You can use this map to plan your job search and identify potential locations for interviews.

If you'd like to explore more job listings or get recommendations on companies to apply to, feel free to ask!

USER: Write a Python script to scrape Amazon product prices.

AGENT: I'm sorry, I can only assist with career-related topics.
